In [1]:
import pandas as pd
from sequence import *
import regex as re
import numpy as np
import pickle

pd.options.display.max_seq_items = 2000


# Load in data sets 

In [2]:
class_1 = readFastaFile('../datasets/kari_ec_1_1_1_86_ec_1_1_1_382_ec_1_1_1_383_swissprot/subsets/classI_kari/kari_ec_1_1_1_86_ec_1_1_1_382_ec_1_1_1_383_swissprot_classI_kari.fasta')
class_2 = readFastaFile('../datasets/kari_ec_1_1_1_86_ec_1_1_1_382_ec_1_1_1_383_swissprot/subsets/classII_kari/kari_ec_1_1_1_86_ec_1_1_1_382_ec_1_1_1_383_swissprot_classII_kari.fasta')

annots = pd.read_csv('../datasets/kari_ec_1_1_1_86_ec_1_1_1_382_ec_1_1_1_383_swissprot/csv/custom/kari_ec_1_1_1_86_ec_1_1_1_382_ec_1_1_1_383_swissprot_annotated.csv')

# Important data columns

In [112]:
annots.loc[:, annots.columns.isin(['accession','ft_binding', 'ft_domain'])]

,accession,ft_binding,ft_domain
0,Q05758,"BINDING 123..130; /ligand=""NADP(+)""; /ligand_i...","DOMAIN 102..300; /note=""KARI N-terminal Rossma..."
1,Q65XK0,"BINDING 111..118; /ligand=""NADP(+)""; /ligand_i...","DOMAIN 90..288; /note=""KARI N-terminal Rossman..."
2,P05793,"BINDING 45..48; /ligand=""NADP(+)""; /ligand_id=...","DOMAIN 15..208; /note=""KARI N-terminal Rossman..."
3,P06168,"BINDING 84..93; /ligand=""NADP(+)""; /ligand_id=...","DOMAIN 57..246; /note=""KARI N-terminal Rossman..."
4,P05989,"BINDING 45..48; /ligand=""NADP(+)""; /ligand_id=...","DOMAIN 15..208; /note=""KARI N-terminal Rossman..."
...,...,...,...
645,Q9AQ97,"BINDING 11; /ligand=""NADP(+)""; /ligand_id=""ChE...","DOMAIN <1..141; /note=""KARI N-terminal Rossman..."
646,Q9K8E7,"BINDING 25..28; /ligand=""NADP(+)""; /ligand_id=...","DOMAIN 2..181; /note=""KARI N-terminal Rossmann..."
647,Q9RQ47,"BINDING 45..48; /ligand=""NADP(+)""; /ligand_id=...","DOMAIN 15..208; /note=""KARI N-terminal Rossman..."
648,Q9RQ51,"BINDING 44..47; /ligand=""NADP(+)""; /ligand_id=...","DOMAIN 16..207; /note=""KARI N-terminal Rossman..."


# Testing regular expressions to extract info

In [114]:
domain = 'DOMAIN (.*?); */note=\"(.*?)";'
metal = 'BINDING (.*?); /ligand=\"(.*?)";'

seq = annots.iloc[:1, :]
seq

,accession,id,gene_names,gene_primary,gene_synonym,gene_oln,gene_orf,organism_name,organism_id,protein_name,...,BRENDA_TN_Hydroxypyruvate_UNITS,BRENDA_TN_Hydroxypyruvate_REFS,BRENDA_TN_Hydroxypyruvate_COMMENT,BRENDA_SY_COMMENT,Non_AA_Character,AA_Character,UniProt_DB,Length_2,Cross_reference_InterPro_2,KARI_Class
0,Q05758,ILV5_ARATH,At3g58610 F14P22.200,NaN,NaN,At3g58610,F14P22.200,Arabidopsis thaliana (Mouse-ear cress),3702,"Ketol-acid reductoisomerase, chloroplastic (EC...",...,NaN,NaN,NaN,NaN,False,True,NaN,591,IPR008927;IPR013328;IPR013023;IPR000506;IPR013...,Class_2


In [9]:
for x in seq['ft_domain']:
    print(x)

DOMAIN 102..300; /note="KARI N-terminal Rossmann"; /evidence="ECO:0000255|PROSITE-ProRule:PRU01197"; DOMAIN 301..449; /note="KARI C-terminal knotted 1"; /evidence="ECO:0000255|PROSITE-ProRule:PRU01198"; DOMAIN 450..586; /note="KARI C-terminal knotted 2"; /evidence="ECO:0000255|PROSITE-ProRule:PRU01198"


In [54]:
for x in seq['ft_binding']:
    print(x)

BINDING 123..130; /ligand="NADP(+)"; /ligand_id="ChEBI:CHEBI:58349"; /evidence="ECO:0000250"; BINDING 156..161; /ligand="NADP(+)"; /ligand_id="ChEBI:CHEBI:58349"; /evidence="ECO:0000250"; BINDING 195..199; /ligand="NADP(+)"; /ligand_id="ChEBI:CHEBI:58349"; /evidence="ECO:0000250"; BINDING 309; /ligand="Mg(2+)"; /ligand_id="ChEBI:CHEBI:18420"; /ligand_label="1"; /evidence="ECO:0000255|PROSITE-ProRule:PRU01198"; BINDING 309; /ligand="Mg(2+)"; /ligand_id="ChEBI:CHEBI:18420"; /ligand_label="2"; /evidence="ECO:0000255|PROSITE-ProRule:PRU01198"; BINDING 313; /ligand="Mg(2+)"; /ligand_id="ChEBI:CHEBI:18420"; /ligand_label="1"; /evidence="ECO:0000255|PROSITE-ProRule:PRU01198"; BINDING 486; /ligand="Mg(2+)"; /ligand_id="ChEBI:CHEBI:18420"; /ligand_label="2"; /evidence="ECO:0000255|PROSITE-ProRule:PRU01198"; BINDING 490; /ligand="Mg(2+)"; /ligand_id="ChEBI:CHEBI:18420"; /ligand_label="2"; /evidence="ECO:0000255|PROSITE-ProRule:PRU01198"; BINDING 512; /ligand="substrate"; /evidence="ECO:0000255|P

In [106]:
exp = re.findall(domain, str(seq['ft_domain'].values))
met = re.findall(metal, str(seq['ft_binding'].values))

In [111]:
met

[('123..130', 'NADP(+)'),
 ('156..161', 'NADP(+)'),
 ('195..199', 'NADP(+)'),
 ('309', 'Mg(2+)'),
 ('309', 'Mg(2+)'),
 ('313', 'Mg(2+)'),
 ('486', 'Mg(2+)'),
 ('490', 'Mg(2+)'),
 ('512', 'substrate')]

In [99]:
exp

[('102..300', 'KARI N-terminal Rossmann'),
 ('301..449', 'KARI C-terminal knotted 1'),
 ('450..586', 'KARI C-terminal knotted 2')]

# Identifying where cofactor residues relative to domains

In [193]:
def sort_features(domains: list) -> dict:
    
    RANGE = 0
    ID = 1
    domain_info = {}
    
    for domain in domains: 
        
        if domain[ID] not in domain_info.keys():
            
            domain_info[domain[ID]] = [domain[RANGE]]
        else:
           
            if domain[RANGE] not in domain_info[domain[ID]]:
                
                domain_info[domain[ID]].append(domain[RANGE])
    
    return domain_info

In [194]:

def summarise_annots(sequences: list):
    """
    Returns a dictionary that has the accession number as key.
    The value is a list, the first element contains a dict with
    all the domains(C or N). Second element contains all cofactors. 
    """
    annotations = {}

    for seq in sequences:    

        entry = annots.loc[annots.accession == seq.name]

        doms = re.findall(domain, str(entry['ft_domain'].values))
        met = re.findall(metal, str(entry['ft_binding'].values))
        
        sorted_doms = sort_features(doms)
        sorted_met = sort_features(met)
        
        info = [sort_features(doms), sort_features(met)]

        annotations[seq.name] = info
    
    
    return annotations


In [219]:
class_1_dict = summarise_annots(class_1)
class_2_dict = summarise_annots(class_2)

In [224]:
outfile_1 = open('class_1_swissprot_annots', 'wb')
pickle.dump(class_1_dict, outfile_1)
outfile_1.close()

outfile_2 = open('class_2_swissprot_annots', 'wb')
pickle.dump(class_2_dict, outfile_2)
outfile_2.close()